# completions

> Otherwise known as *chat completions*. See the [`litellm` documention](https://docs.litellm.ai/docs/completion).

In [ ]:
#|default_exp llm.completions

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()

In [ ]:
#|export
try:
    import litellm
    import inspect
    import functools
    from typing import List
    from adulib.llm._utils import _llm_func_factory, _llm_async_func_factory
    from adulib.llm.tokens import token_counter
except ImportError as e:
    raise ImportError(f"Install adulib[llm] to use this API.") from e

In [ ]:
#|hide
from pydantic import BaseModel
from adulib.caching import set_default_cache_path
import adulib.llm.completions as this_module

In [ ]:
#|hide
repo_path = nblite.config.get_project_root_and_config()[0]
set_default_cache_path(repo_path / '.tmp_cache')

The two required fields for completions are `model` and `message`. Some optional arguments are:

#### Properties of `messages`

Each message in the `messages` array can include the following fields:

- `role: str` (**required**) - The role of the message's author. Roles can be: system, user, assistant, function or tool.
- `content: Union[str,List[dict],None]` (**required**) - The contents of the message. It is required for all messages, but may be null for assistant messages with function calls.
- `name: str` - The name of the author of the message. It is required if the role is "function". The name should match the name of the function represented in the content. It can contain characters (a-z, A-Z, 0-9), and underscores, with a maximum length of 64 characters.
- `function_call: object` - The name and arguments of a function that should be called, as generated by the model.
- `tool_call_id: str` - Tool call that this message is responding to.


#### Explanation of roles

- **system**: Sets assistant context. Example: `{ "role": "system", "content": "You are a helpful assistant." }`
- **user**: End user input. Example:  `{ "role": "user", "content": "What's the weather like today?" }`
- **assistant**: AI response. Example: `{ "role": "assistant", "content": "The weather is sunny and warm." }`
- **function**: Function call/result (`name` required). Example:  `{ "role": "function", "name": "get_weather", "content": "{\"location\": \"San Francisco\"}" }`
- **tool**: Tool/plugin interaction (`tool_call_id` required). Example: `{ "role": "tool", "tool_call_id": "abc123", "content": "Tool response here" }`

In [ ]:
#|echo: false
show_doc(this_module.completion)

## completion

```python
completion(
   *args,
   cache_enabled: bool,
   cache_path: typing.Union[str, pathlib.Path, NoneType],
   cache_key_prefix: typing.Optional[str],
   include_model_in_cache_key: bool,
   return_cache_key: bool,
   **kwargs
)
```

This function is a wrapper around a corresponding function in the `litellm` library, see [this](https://docs.litellm.ai/docs/completion/input) for a full list of the available arguments.

---


In [ ]:
#|export
completion = _llm_func_factory(
    func=litellm.completion,
    func_name="completion",
    func_cache_name="completion",
    retrieve_log_data=lambda model, func_kwargs, response: {
        "method": "completion",
        "input_tokens": token_counter(model=model, messages=func_kwargs['messages']),
        "output_tokens": sum([token_counter(model=model, messages=[{'role': c.message.role, 'content': c.message.content}]) for c in response.choices]),
        "cost": response._hidden_params['response_cost'],
    }
)

completion.__doc__ = """
This function is a wrapper around a corresponding function in the `litellm` library, see [this](https://docs.litellm.ai/docs/completion/input) for a full list of the available arguments.
""".strip()

In [ ]:
response = completion(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
)
response.choices[0].message.content

'The capital of France is Paris.'

In [ ]:
class Recipe(BaseModel):
    name: str
    ingredients: List[str]
    steps: List[str]

response = completion(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful cooking assistant."},
        {"role": "user", "content": "Give me a simple recipe for pancakes."}
    ],
    response_format=Recipe
)

Recipe.model_validate_json(response.choices[0].message.content).model_dump()

{'name': 'Simple Pancakes',
 'ingredients': ['1 cup all-purpose flour',
  '2 tablespoons sugar',
  '2 teaspoons baking powder',
  '1/2 teaspoon salt',
  '1 cup milk',
  '1 egg',
  '2 tablespoons melted butter',
  '1 teaspoon vanilla extract'],
 'steps': ['In a large mixing bowl, whisk together the flour, sugar, baking powder, and salt.',
  'In another bowl, combine the milk, egg, melted butter, and vanilla extract.',
  'Pour the wet ingredients into the dry ingredients and stir until just combined. (A few lumps are okay.)',
  'Preheat a non-stick skillet or griddle over medium heat. Lightly grease with butter or oil if needed.',
  'Pour about 1/4 cup of batter onto the skillet for each pancake.',
  'Cook until bubbles form on the surface and the edges look set, about 2-3 minutes.',
  'Flip and cook for another 1-2 minutes until golden brown.',
  'Serve warm with your favorite toppings such as syrup, fruit, or whipped cream.']}

You can save costs during testing using [mock responses](https://docs.litellm.ai/docs/completion/mock_requests):

In [ ]:
response = completion(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Sweden?"}
    ],
    mock_response = "Stockholm"
)
response.choices[0].message.content

'Stockholm'

In [ ]:
#|echo: false
show_doc(this_module.async_completion)

## async_completion *(async)*

```python
async_completion(
   model: str,
   *args,
   cache_enabled: bool,
   cache_path: typing.Union[str, pathlib.Path, NoneType],
   cache_key_prefix: typing.Optional[str],
   include_model_in_cache_key: bool,
   return_cache_key: bool,
   **kwargs
)
```

---


In [ ]:
#|export
async_completion = _llm_async_func_factory(
    func=litellm.acompletion,
    func_name="async_completion",
    func_cache_name="completion",
    retrieve_log_data=lambda model, func_kwargs, response: {
        "method": "completion",
        "input_tokens": token_counter(model=model, messages=func_kwargs['messages']),
        "output_tokens": sum([token_counter(model=model, messages=[{'role': c.message.role, 'content': c.message.content}]) for c in response.choices]),
        "cost": response._hidden_params['response_cost'],
    }
)

completion.__doc__ = """
This function is a wrapper around a corresponding function in the `litellm` library, see [this](https://docs.litellm.ai/docs/completion/input) for a full list of the available arguments.
""".strip()

In [ ]:
response = await async_completion(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ],
)
response.choices[0].message.content

TypeError: missing a required argument: 'model'

In [ ]:
#|echo: false
show_doc(this_module.prompt)

## async_prompt

```python
async_prompt(
   model: str,
   *args,
   cache_enabled: bool,
   cache_path: typing.Union[str, pathlib.Path, NoneType],
   cache_key_prefix: typing.Optional[str],
   include_model_in_cache_key: bool,
   return_cache_key: bool,
   **kwargs
)
```

Simplified chat completions designed for single-turn tasks like classification, summarization, or extraction. For a full list of the available arguments see the [documentation](https://docs.litellm.ai/docs/completion/input) for the `completion` function in `litellm`.

---


In [ ]:
#|export
@functools.wraps(completion)
def prompt(
    model: str,
    context: str,
    prompt: str,
    *args,
    return_full_response: bool=False,
    **kwargs,
):
    messages = [
        {"role": "system", "content": context},
        {"role": "user", "content": prompt}
    ]
    response = completion(model, messages, *args, **kwargs)
    if return_full_response: return response
    else: return response.choices[0].message.content
    
sig = inspect.signature(completion)
prompt.__signature__ = sig.replace(parameters=[p for p in sig.parameters.values() if p.name != 'messages'])
prompt.__name__ = "prompt"
prompt.__doc__ = """
Simplified chat completions designed for single-turn tasks like classification, summarization, or extraction. For a full list of the available arguments see the [documentation](https://docs.litellm.ai/docs/completion/input) for the `completion` function in `litellm`.
""".strip()

In [ ]:
prompt(
    model='gpt-4o-mini',
    context='You are a helpful assistant.',
    prompt='What is the capital of France?',
)

'The capital of France is Paris.'

In [ ]:
#|echo: false
show_doc(this_module.async_prompt)

## completion *(async)*

```python
completion(
   model: str,
   *args,
   cache_enabled: bool,
   cache_path: typing.Union[str, pathlib.Path, NoneType],
   cache_key_prefix: typing.Optional[str],
   include_model_in_cache_key: bool,
   return_cache_key: bool,
   **kwargs
)
```

This function is a wrapper around a corresponding function in the `litellm` library, see [this](https://docs.litellm.ai/docs/completion/input) for a full list of the available arguments.

---


In [ ]:
#|export
@functools.wraps(completion)
async def async_prompt(
    model: str,
    context: str,
    prompt: str,
    *args,
    return_full_response: bool=False,
    **kwargs,
):
    messages = [
        {"role": "system", "content": context},
        {"role": "user", "content": prompt}
    ]
    response = await async_completion(model, messages, *args, **kwargs)
    if return_full_response: return response
    else: return response.choices[0].message.content
    
sig = inspect.signature(completion)
prompt.__signature__ = sig.replace(parameters=[p for p in sig.parameters.values() if p.name != 'messages'])
prompt.__name__ = "async_prompt"
prompt.__doc__ = """
Simplified chat completions designed for single-turn tasks like classification, summarization, or extraction. For a full list of the available arguments see the [documentation](https://docs.litellm.ai/docs/completion/input) for the `completion` function in `litellm`.
""".strip()